In [ ]:
%matplotlib inline
from matplotlib import pylab as plt
from ipywidgets import interact
import ipywidgets as widgets

In [ ]:
from geonotebook.wrappers import RasterDataCollection
import os

In [ ]:
DATA_DIR="/data/kotfic/NEX/golden_tile_layer/WELD/golden_tiles/geotiff/NBAR/"

In [ ]:
def sort_NBAR(a, b):
  am, ay = int(a.split(".")[2][-2:]), int(a.split(".")[3])
  bm, by = int(b.split(".")[2][-2:]), int(b.split(".")[3])

  if ay < by:
    return -1
  elif ay > by:
    return 1
  elif by == ay:
    if am < bm:
      return -1
    elif am > bm:
      return 1
    else:
      return 0
  

PATHS = [DATA_DIR + p for p in sorted(os.listdir(DATA_DIR), sort_NBAR)]

In [ ]:
rdc = RasterDataCollection(PATHS)

In [ ]:
M.add_layer(rdc[:,[1,2,3]], 'NBAR', opacity=0.8)

In [ ]:
# M.remove_layer(M.layers[-1].name)

In [ ]:
TIME_STEP = 10

def ndvi_colormap(numcolors=11, name='custom_div_cmap',
                  mincol='blue', midcol='beige', maxcol='green'):
    from matplotlib.colors import LinearSegmentedColormap 
    return LinearSegmentedColormap.from_list(name=name, 
                                             colors =[mincol, midcol, maxcol],
                                             N=numcolors)

plt.figure(1, figsize=(20., 12.))
plt.imshow(rdc[TIME_STEP,4].get_data(), 
             interpolation='none', cmap=ndvi_colormap())
plt.title(rdc[TIME_STEP,4].name)
plt.show()

In [ ]:
rdc[0,4].get_data()

In [ ]:
from skimage.exposure import rescale_intensity
fig, ax = plt.subplots(figsize=(12, 12))
ax.imshow(rescale_intensity(rdc[0,].get_data()), interpolation='none')